In [75]:
import json

In [76]:
f = open('model2.json')

model = json.load(f)

f.close()

model

{'meta': {'name': 'Cloud Description',
  'description': 'Описание признаков облаков',
  'schemaVersion': '2'},
 'schema': [{'param': 'shining',
   'title': 'Солнце и Луна просвечивают',
   'description': 'Видно ли Солнце или Луну сквозь облака',
   'spec': {'type': 'bool'},
   'require': True},
  {'param': 'height',
   'title': 'Высота облаков',
   'description': 'Оценочная высота облаков в метрах',
   'spec': {'type': 'enum',
    'data': [{'title': 'Очень высоко',
      'description': 'Облака находятся очень высоко над землей',
      'image': 'sample.jpg',
      'value': 'very_high'},
     {'title': 'Высоко',
      'description': 'Облака находятся высоко над землей',
      'image': 'sample.jpg',
      'value': 'high'},
     {'title': 'Средне',
      'description': 'Облака находятся средне над землей, ни высоко, ни низко',
      'image': 'sample.jpg',
      'value': 'middle'},
     {'title': 'Низко',
      'description': 'Облака находятся низко над землей',
      'image': 'sample.jpg',

In [77]:
f = open('data2.json')

data = json.load(f)

f.close()

data

{'meta': {'name': 'Cloud Classificator',
  'description': 'Классификатор облаков на основании признаков',
  'schemaVersion': '2'},
 'stack': [{'id': 'type',
   'title': 'Тип облаков',
   'description': 'Тип облака определяется условием образования',
   'limit': 1},
  {'id': 'morphology',
   'title': 'Морфологическая классификация',
   'description': 'На основании структуры и внешнего вида облака'},
  {'id': 'attention',
   'title': 'Предупреждение о опасностях',
   'description': 'Возможные опасные погодные явления, сопутствующие данному виду облаков'}],
 'graph': [{'type': 'condition',
   'comment': 'Это просвечивающиеся облака и средние?',
   'statements': [{'type': 'statement',
     'param': 'shining',
     'predicate': '=',
     'value': True,
     'require': False,
     'score': 1},
    {'type': 'statement',
     'param': 'height',
     'predicate': '=',
     'value': 'middle',
     'require': False,
     'score': 1}],
   'graph': [{'type': 'report',
     'stack': 'type',
     're

In [78]:
obj = {'shining': True,
 'height': 'middle'}


for param in model['schema']:
    if param['param'] not in obj:
        obj[param['param']] = None

print(obj)

{'shining': True, 'height': 'middle', 'visual_morph': None, 'color': None, 'speed': None}


In [79]:
obj = {}

for param in model['schema']:
    obj[param['param']] = None

obj

{'shining': None,
 'height': None,
 'visual_morph': None,
 'color': None,
 'speed': None}

In [80]:
print(model)

{'meta': {'name': 'Cloud Description', 'description': 'Описание признаков облаков', 'schemaVersion': '2'}, 'schema': [{'param': 'shining', 'title': 'Солнце и Луна просвечивают', 'description': 'Видно ли Солнце или Луну сквозь облака', 'spec': {'type': 'bool'}, 'require': True}, {'param': 'height', 'title': 'Высота облаков', 'description': 'Оценочная высота облаков в метрах', 'spec': {'type': 'enum', 'data': [{'title': 'Очень высоко', 'description': 'Облака находятся очень высоко над землей', 'image': 'sample.jpg', 'value': 'very_high'}, {'title': 'Высоко', 'description': 'Облака находятся высоко над землей', 'image': 'sample.jpg', 'value': 'high'}, {'title': 'Средне', 'description': 'Облака находятся средне над землей, ни высоко, ни низко', 'image': 'sample.jpg', 'value': 'middle'}, {'title': 'Низко', 'description': 'Облака находятся низко над землей', 'image': 'sample.jpg', 'value': 'low'}, {'title': 'Очень низко', 'description': 'Облака находятся очень низко над землей', 'image': 'sa

In [81]:
obj['height'] = 'middle'
# obj['ground_temperature'] = 50
obj['color'] = 'bluish'
# obj['visual_morph'] = 'wavy'
obj['shining'] = True
obj

{'shining': True,
 'height': 'middle',
 'visual_morph': None,
 'color': 'bluish',
 'speed': None}

In [82]:
stack = {}

def stack_flush():
    for s in data['stack']:
        stack[s['id']] = []

def stack_result(id, result, score, explain, bypass):
    result['score'] = score
    result['explain'] = explain
    result['bypass'] = bypass
    stack[id].append(result)

In [83]:
class Node():
    def __init__(self, graph, depth = 0):
        self.type = graph['type']
        self.graph = graph
        self.nodes = []
        self.depth = depth
        self.score = 0
        
        try:
            self.stack = graph['stack']
        except:
            self.stack = None
        
        try:
            self.comment = graph['comment']
        except:
            self.comment = None

        try:
            for g in graph['graph']:
                self.nodes.append(Node(g, depth + 1))
        except:
            pass

        match self.type:
            case 'condition':
                self.statements = graph['statements']
            case 'report':
                self.report = graph['report']

    def debug_trace(self, trace = []):
        trace.append(self)
        for node in self.nodes:
            node.debug_trace(trace)

        return trace
    
    def debug_info(self):
        return {
            'node_count': len(self.nodes), 
            'depth': self.depth,
            'type': self.type
        }
    
    def debug_tree(self):
        for n in self.debug_trace():
            info = n.debug_info()
            print('     ' * n.depth + str(info))

    def execute_condition(self):
        true_count = 0
        global_score = 0
        explain = []
        bypass = []

        for statement in self.statements:
            try:
                st_score = statement['score']
            except:
                st_score = 0

            if statement['param'] in obj:
                if obj[statement['param']] is not None:
                    values = []
                    if type(statement['value']) is not list:
                        values.append(statement['value'])
                    else:
                        values = statement['value']
                    
                    for dst in values:
                        ost = obj[statement['param']]

                        match statement['predicate']:
                            case '>':
                                if float(ost) > float(dst):
                                    true_count += 1
                                    global_score += st_score
                                    explain.append(statement)
                            case '<':
                                if float(ost) < float(dst):
                                    true_count += 1
                                    global_score += st_score
                                    explain.append(statement)
                            case '=':
                                if ost == dst:
                                    true_count += 1
                                    global_score += st_score
                                    explain.append(statement)
                            case '>=':
                                if float(ost) >= float(dst):
                                    true_count += 1
                                    global_score += st_score
                                    explain.append(statement)
                            case '<=':
                                if float(ost) <= float(dst):
                                    true_count += 1
                                    global_score += st_score
                                    explain.append(statement)
                            case '!=':
                                if float(ost) != float(dst):
                                    true_count += 1
                                    global_score += st_score
                                    explain.append(statement)
                            case _:
                                pass
                else:
                    if not statement['require']:
                        true_count += 1
                        bypass.append(statement)
        
        if true_count == len(self.statements):
            self.execute_nodes(global_score + self.score, explain, bypass)

    def execute(self, explain = [], score = 0, bypass = []):
        self.score = score

        match self.type:
            case 'report':
                if 'requireScore' in self.report:
                    if self.report['requireScore'] <= self.score:
                        stack_result(self.stack, self.report, self.score, explain, bypass)
                        self.execute_nodes(self.score)
                else:
                    stack_result(self.stack, self.report, self.score, explain, bypass)
                    self.execute_nodes(self.score)
            case 'condition':
                self.execute_condition()
    
    def execute_nodes(self, score, explain = [], bypass = []):
        for n in self.nodes:
            n.execute(explain, score, bypass)


In [84]:
stack_flush()

for i in data['graph']:
    n = Node(i)
    n.debug_tree()
    n.execute()

print(json.dumps(stack, indent=4, ensure_ascii=False))

{'node_count': 4, 'depth': 0, 'type': 'condition'}
     {'node_count': 0, 'depth': 1, 'type': 'report'}
     {'node_count': 1, 'depth': 1, 'type': 'condition'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}
     {'node_count': 1, 'depth': 1, 'type': 'condition'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}
     {'node_count': 4, 'depth': 1, 'type': 'condition'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}
          {'node_count': 1, 'depth': 2, 'type': 'condition'}
               {'node_count': 0, 'depth': 3, 'type': 'report'}
          {'node_count': 1, 'depth': 2, 'type': 'condition'}
               {'node_count': 0, 'depth': 3, 'type': 'report'}
          {'node_count': 1, 'depth': 2, 'type': 'condition'}
               {'node_count': 0, 'depth': 3, 'type': 'report'}
{'node_count': 4, 'depth': 0, 'type': 'condition'}
     {'node_count': 0, 'depth': 1, 'type': 'report'}
     {'node_count': 1, 'depth': 1, 'type': 'condition'}
          {'node_count

In [85]:
n = Node(i)
n.execute()

print(json.dumps(stack, indent=4, ensure_ascii=False))

{
    "type": [
        {
            "title": "Облака среднего яруса",
            "description": "Все облака среднего яруса имеют смешанную структуру из смеси капелек с ледяными кристаллами. Облака среднего яруса обычно встречаются на высотах 2-7 км для умеренных широт, 2-4 км - для полярных и 2-8 км - для тропических. Поскольку их высоты небольшие, то они преимущественно состоят из капелек воды, но зимой, да еще в наших широтах, конечно, содержат кристаллики льда. Осадки, выпадающие из них летом, поверхности земли не достигают.",
            "requireScore": 2,
            "score": 2,
            "explain": [
                {
                    "type": "statement",
                    "param": "shining",
                    "predicate": "=",
                    "value": true,
                    "require": false,
                    "score": 1
                },
                {
                    "type": "statement",
                    "param": "height",
                    "pr